In [1]:
from bqplot import *

import numpy as np

## ColorScale

The colors for the `ColorScale` can be defined one of two ways:
- Manually, by setting the scale's `colors` attribute to a list of css colors. They can be either:
   - html colors (link) `'white'`
   - hex `'#000000'`
   - rgb `'rgb(0, 0, 0)'`.

```python
col_sc = ColorScale(colors=['yellow', 'red'])
```
- Using one of `bqplot`'s color-schemes. As of now we support all the colorbrewer schemes (link), as well as the matplotlib schemes 'viridis', 'magma', 'inferno' and 'plasma'.
```python
col_sc = ColorScale(scheme=['viridis'])
```

The color scale then linearly interpolates between its colors.

### ColorAxis

A `ColorAxis`, like other `Axis` types, takes a color scale as input. It can then be displayed in a `Figure`.

```python
ax_col = ColorAxis(scale=col_sc)
```

In [2]:
size = 100
np.random.seed(0)
x_data = range(size)
y_data = np.cumsum(np.random.randn(size) * 100.0)

def create_figure(color_scale, color_data):

    scales = {'x': LinearScale(), 'y': LinearScale(), 'color': color_scale}
    scatter = Scatter(x=x_data, y=y_data, color=color_data,
                      scales=scales, 
                      stroke='black')
    ax_y = Axis(label='Price', scale=scales['y'], 
                orientation='vertical', side='left')
    ax_x = Axis(label='Day', scale=scales['x'], num_ticks=10, label_location='end')
    ax_c = ColorAxis(scale=color_scale, label='Returns', orientation='vertical', side='right')
    ax_y = Axis(scale=scales['y'], 
                orientation='vertical', side='left')
    ax_x = Axis(scale=scales['x'], num_ticks=10, label_location='end')
    ax_c = ColorAxis(scale=color_scale, orientation='vertical', side='right')

    m_chart = dict(top=50, bottom=70, left=50, right=100)

    return Figure(axes=[ax_x, ax_y, ax_c], title='Up and Down',
                  marks=[scatter], fig_margin=m_chart,)

col_sc = ColorScale()
create_figure(col_sc, y_data)

Figure(axes=[Axis(label_location='end', num_ticks=10, scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale(), side='left'), ColorAxis(orientation='vertical', scale=ColorScale(), side='right')], fig_margin={'top': 50, 'right': 100, 'left': 50, 'bottom': 70}, layout=Layout(min_width=u'125px'), marks=[Scatter(color=array([  176.4052346 ,   216.42095543,   314.29475384,   538.38407376,
         725.13987278,   627.41208479,   722.42092654,   707.28520571,
         696.96332053,   738.02317073,   752.42752784,   897.85487854,
         973.95865106,   986.12615271,  1030.51247598,  1063.87990872,
        1213.28781603,  1192.77198966,  1224.07875982,  1138.66918589,
         883.37020431,   948.73206385,  1035.17568374,   960.9591817 ,
        1187.9346441 ,  1042.49807664,  1047.07392837,  1028.35554336,
        1181.6334648 ,  1328.56934179,  1344.06408436,  1381.88033632,
        1293.10176156,  1095.02211473,  1060.2308998 ,  1075.86579671,
        1198.89486478,  1319.13284966,  1280.40016792,  1250.16989286,
        1145.31459636,  1003.31280264,   832.68578358,  1027.7633231 ,
         976.79810493,   932.99067476,   807.71113876,   885.46017434,
         724.07038959,   702.79636157,   613.24970545,   651.93995523,
         600.85944148,   482.79622306,   479.97800023,   522.81118728,
         529.46290952,   559.71009929,   496.27788993,   460.00377333,
         392.75772855,   356.8024124 ,   275.48778419,   102.85952396,
         120.60213818,    80.42404456,   -82.59579013,   -36.31756458,
        -127.04740102,  -121.85286144,   -48.94380522,   -36.04551415,
          77.89455431,   -45.58802773,    -5.35386361,   -73.8348727 ,
        -160.91458762,  -218.7995541 ,  -249.95480731,  -244.33827309,
        -360.85325717,  -270.77060847,  -224.2043645 ,  -377.82873313,
        -229.00351375,   -39.41459614,    78.46336097,    60.47087739,
         -46.60438476,    58.84078793,    18.52309324,   140.76760027,
         161.59509808,   259.25900173,   294.89564145,   365.55295827,
         366.60296034,   545.19000973,   557.881219  ,   598.08015534]), colors=['steelblue'], interactions={'hover': 'tooltip'}, scales={'y': LinearScale(), 'x': LinearScale(), 'color': ColorScale()}, scales_metadata={'opacity': {'dimension': 'opacity'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'skew': {'dimension': 'skew'}, 'rotation': {'dimension': 'rotation'}, 'size': {'dimension': 'size'}}, stroke='black', tooltip_style={'opacity': 0.9}, x=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), y=array([  176.4052346 ,   216.42095543,   314.29475384,   538.38407376,
         725.13987278,   627.41208479,   722.42092654,   707.28520571,
         696.96332053,   738.02317073,   752.42752784,   897.85487854,
         973.95865106,   986.12615271,  1030.51247598,  1063.87990872,
        1213.28781603,  1192.77198966,  1224.07875982,  1138.66918589,
         883.37020431,   948.73206385,  1035.17568374,   960.9591817 ,
        1187.9346441 ,  1042.49807664,  1047.07392837,  1028.35554336,
        1181.6334648 ,  1328.56934179,  1344.06408436,  1381.88033632,
        1293.10176156,  1095.02211473,  1060.2308998 ,  1075.86579671,
        1198.89486478,  1319.13284966,  1280.40016792,  1250.16989286,
        1145.31459636,  1003.31280264,   832.68578358,  1027.7633231 ,
         976.79810493,   932.99067476,   807.71113876,   885.46017434,
         724.07038959,   702.79636157,   613.24970545,   651.93995523,
         600.85944148,   482.79622306,   479.97800

### Attributes
`ColorScales` share attributes with the other `Scale` types:
- Their domain can be manually constrained with the `min` and `max` attributes
- They can be inversed by setting the `reverse` attribute to `True`

In [3]:
col_sc.min = 0

In [4]:
col_sc.reverse = True

#### Mid
In addition they also have a `mid` attribute, a value that will be mapped to the middle color. This is especially suited to diverging color schemes.

In [5]:
col_sc.min = None
col_sc.mid = 0

## DateColorScale

The `DateColorScale` is a color scale for dates. It works in every way like the regular `ColorScale`, except that its `min`, `mid` and `max` attributes — if defined — must be date elements (`datetime`, `numpy` or `pandas`).

In [6]:
import pandas as pd

date_col_sc = DateColorScale()
dates = pd.date_range(start='01-01-2007', periods=size)
create_figure(date_col_sc, dates)

Figure(axes=[Axis(label_location='end', num_ticks=10, scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale(), side='left'), ColorAxis(orientation='vertical', scale=DateColorScale(), side='right')], fig_margin={'top': 50, 'right': 100, 'left': 50, 'bottom': 70}, layout=Layout(min_width=u'125px'), marks=[Scatter(color=array(['2007-01-01T00:00:00.000000000', '2007-01-02T00:00:00.000000000',
       '2007-01-03T00:00:00.000000000', '2007-01-04T00:00:00.000000000',
       '2007-01-05T00:00:00.000000000', '2007-01-06T00:00:00.000000000',
       '2007-01-07T00:00:00.000000000', '2007-01-08T00:00:00.000000000',
       '2007-01-09T00:00:00.000000000', '2007-01-10T00:00:00.000000000',
       '2007-01-11T00:00:00.000000000', '2007-01-12T00:00:00.000000000',
       '2007-01-13T00:00:00.000000000', '2007-01-14T00:00:00.000000000',
       '2007-01-15T00:00:00.000000000', '2007-01-16T00:00:00.000000000',
       '2007-01-17T00:00:00.000000000', '2007-01-18T00:00:00.000000000',
       '2007-01-19T00:00:00.000000000', '2007-01-20T00:00:00.000000000',
       '2007-01-21T00:00:00.000000000', '2007-01-22T00:00:00.000000000',
       '2007-01-23T00:00:00.000000000', '2007-01-24T00:00:00.000000000',
       '2007-01-25T00:00:00.000000000', '2007-01-26T00:00:00.000000000',
       '2007-01-27T00:00:00.000000000', '2007-01-28T00:00:00.000000000',
       '2007-01-29T00:00:00.000000000', '2007-01-30T00:00:00.000000000',
       '2007-01-31T00:00:00.000000000', '2007-02-01T00:00:00.000000000',
       '2007-02-02T00:00:00.000000000', '2007-02-03T00:00:00.000000000',
       '2007-02-04T00:00:00.000000000', '2007-02-05T00:00:00.000000000',
       '2007-02-06T00:00:00.000000000', '2007-02-07T00:00:00.000000000',
       '2007-02-08T00:00:00.000000000', '2007-02-09T00:00:00.000000000',
       '2007-02-10T00:00:00.000000000', '2007-02-11T00:00:00.000000000',
       '2007-02-12T00:00:00.000000000', '2007-02-13T00:00:00.000000000',
       '2007-02-14T00:00:00.000000000', '2007-02-15T00:00:00.000000000',
       '2007-02-16T00:00:00.000000000', '2007-02-17T00:00:00.000000000',
       '2007-02-18T00:00:00.000000000', '2007-02-19T00:00:00.000000000',
       '2007-02-20T00:00:00.000000000', '2007-02-21T00:00:00.000000000',
       '2007-02-22T00:00:00.000000000', '2007-02-23T00:00:00.000000000',
       '2007-02-24T00:00:00.000000000', '2007-02-25T00:00:00.000000000',
       '2007-02-26T00:00:00.000000000', '2007-02-27T00:00:00.000000000',
       '2007-02-28T00:00:00.000000000', '2007-03-01T00:00:00.000000000',
       '2007-03-02T00:00:00.000000000', '2007-03-03T00:00:00.000000000',
       '2007-03-04T00:00:00.000000000', '2007-03-05T00:00:00.000000000',
       '2007-03-06T00:00:00.000000000', '2007-03-07T00:00:00.000000000',
       '2007-03-08T00:00:00.000000000', '2007-03-09T00:00:00.000000000',
       '2007-03-10T00:00:00.000000000', '2007-03-11T00:00:00.000000000',
       '2007-03-12T00:00:00.000000000', '2007-03-13T00:00:00.000000000',
       '2007-03-14T00:00:00.000000000', '2007-03-15T00:00:00.000000000',
       '2007-03-16T00:00:00.000000000', '2007-03-17T00:00:00.000000000',
       '2007-03-18T00:00:00.000000000', '2007-03-19T00:00:00.000000000',
       '2007-03-20T00:00:00.000000000', '2007-03-21T00:00:00.000000000',
       '2007-03-22T00:00:00.000000000', '2007-03-23T00:00:00.000000000',
       '2007-03-24T00:00:00.000000000', '2007-03-25T00:00:00.000000000',
       '2007-03-26T00:00:00.000000000', '2007-03-27T00:00:00.000000000',
       '2007-03-28T00:00:00.000000000', '2007-03-29T00:00:00.000000000',
       '2007-03-30T00:00:00.000000000', '2007-03-31T00:00:00.000000000',
       '2007-04-01T00:00:00.000000000', '2007-04-02T00:00:00.000000000',
       '2007-04-03T00:00:00.000000000', '2007-04-04T00:00:00.000000000',
       '2007-04-05T00:00:00.000000000', '2007-04-06T00:00:00.000000000',
       '2007-04-07T00:00:00.000000000', '2007-04-08T00:00:00.000000000',
       '2007-04-09T00:00:00.000000000', '2007-04-10T00:00:00.000000000'], dtype='datetime

In [7]:
date_col_sc.min = pd.datetime(2007,2,28)

## Color Schemes

Use the following widgets to browse through the available color schemes

In [8]:
from bqplot.market_map import MarketMap

from ipywidgets import IntSlider, SelectionSlider, Dropdown
from ipywidgets import VBox, HBox, Layout
from traitlets import link

### Diverging schemes

In [9]:
div_schemes = [
    'Spectral',
    'RdYlGn',
    'RdBu',
    'PiYG',
    'PRGn',
    'RdYlBu',
    'BrBG',
    'RdGy',
    'PuOr',
]

In [10]:
def scheme_inspector(fig, schemes, title=''):
    '''
    Takes a Figure and a list of schemes and returns the Figure along with
    sliders to go through the different schemes.
    
    Supposes that the 
    '''
    
    # Get the color scale
    col_sc = fig.marks[0].scales['color']
    
    # Create the widgets to select the colorscheme
    scheme_slider = SelectionSlider(description='',
                                    options=schemes)
    scheme_dd = Dropdown(description='Scheme',
                         options=schemes)
    link((scheme_dd, 'value'), (scheme_slider, 'value'))

    def update_scheme(*args):
        col_sc.scheme = scheme_slider.value
        # ax_c.label = scheme_slider.value

    scheme_slider.observe(update_scheme, 'value')
    scheme_dd.observe(update_scheme, 'value')
    
    update_scheme()

    return VBox([HBox([scheme_dd, scheme_slider]), fig])

scheme_inspector(create_figure(ColorScale(), y_data), div_schemes)

VBox(children=(HBox(children=(Dropdown(description=u'Scheme', options=('Spectral', 'RdYlGn', 'RdBu', 'PiYG', 'PRGn', 'RdYlBu', 'BrBG', 'RdGy', 'PuOr'), value='Spectral'), SelectionSlider(options=('Spectral', 'RdYlGn', 'RdBu', 'PiYG', 'PRGn', 'RdYlBu', 'BrBG', 'RdGy', 'PuOr'), value='Spectral'))), Figure(axes=[Axis(label_location='end', num_ticks=10, scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale(), side='left'), ColorAxis(orientation='vertical', scale=ColorScale(scheme=u'Spectral'), side='right')], fig_margin={'top': 50, 'right': 100, 'left': 50, 'bottom': 70}, layout=Layout(min_width=u'125px'), marks=[Scatter(color=array([  176.4052346 ,   216.42095543,   314.29475384,   538.38407376,
         725.13987278,   627.41208479,   722.42092654,   707.28520571,
         696.96332053,   738.02317073,   752.42752784,   897.85487854,
         973.95865106,   986.12615271,  1030.51247598,  1063.87990872,
        1213.28781603,  1192.77198966,  1224.07875982,  1138.66918589,
         883.37020431,   948.73206385,  1035.17568374,   960.9591817 ,
        1187.9346441 ,  1042.49807664,  1047.07392837,  1028.35554336,
        1181.6334648 ,  1328.56934179,  1344.06408436,  1381.88033632,
        1293.10176156,  1095.02211473,  1060.2308998 ,  1075.86579671,
        1198.89486478,  1319.13284966,  1280.40016792,  1250.16989286,
        1145.31459636,  1003.31280264,   832.68578358,  1027.7633231 ,
         976.79810493,   932.99067476,   807.71113876,   885.46017434,
         724.07038959,   702.79636157,   613.24970545,   651.93995523,
         600.85944148,   482.79622306,   479.97800023,   522.81118728,
         529.46290952,   559.71009929,   496.27788993,   460.00377333,
         392.75772855,   356.8024124 ,   275.48778419,   102.85952396,
         120.60213818,    80.42404456,   -82.59579013,   -36.31756458,
        -127.04740102,  -121.85286144,   -48.94380522,   -36.04551415,
          77.89455431,   -45.58802773,    -5.35386361,   -73.8348727 ,
        -160.91458762,  -218.7995541 ,  -249.95480731,  -244.33827309,
        -360.85325717,  -270.77060847,  -224.2043645 ,  -377.82873313,
        -229.00351375,   -39.41459614,    78.46336097,    60.47087739,
         -46.60438476,    58.84078793,    18.52309324,   140.76760027,
         161.59509808,   259.25900173,   294.89564145,   365.55295827,
         366.60296034,   545.19000973,   557.881219  ,   598.08015534]), colors=['steelblue'], interactions={'hover': 'tooltip'}, scales={'y': LinearScale(), 'x': LinearScale(), 'color': ColorScale(scheme=u'Spectral')}, scales_metadata={'opacity': {'dimension': 'opacity'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'skew': {'dimension': 'skew'}, 'rotation': {'dimension': 'rotation'}, 'size': {'dimension': 'size'}}, stroke='black', tooltip_style={'opacity': 0.9}, x=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), y=array([  176.4052346 ,   216.42095543,   314.29475384,   538.38407376,
         725.13987278,   627.41208479,   722.42092654,   707.28520571,
         696.96332053,   738.02317073,   752.42752784,   897.85487854,
         973.95865106,   986.12615271,  1030.51247598,  1063.87990872,
        1213.28781603,  1192.77198966,  1224.07875982,  1138.66918589,
         883.37020431,   948.73206385,  1035.17568374,   960.9591817 ,
        1187.9346441 ,  1042.49807664,  1047.07392837,  1028.35554336,
        1181.6334648 ,  1328.56934179,  1344.06408436,  1381.88033632,
        1293.10176156,  1095.02211473,  1060.2308998 ,  1075.86579671,


### Non-diverging schemes

In [11]:
lin_schemes = [
    'OrRd',
    'PuBu',
    'BuPu',
    'Oranges',
    'BuGn',
    'YlOrBr',
    'YlGn',
    'Reds',
    'RdPu',
    'Greens',
    'YlGnBu',
    'Purples',
    'GnBu',
    'Greys',
    'YlOrRd',
    'PuRd',
    'Blues',
    'PuBuGn',
    'viridis',
    'plasma',
    'inferno',
    'magma'
]

In [12]:
scheme_inspector(create_figure(ColorScale(), y_data), lin_schemes,
                 title='Non-diverging schemes')

VBox(children=(HBox(children=(Dropdown(description=u'Scheme', options=('OrRd', 'PuBu', 'BuPu', 'Oranges', 'BuGn', 'YlOrBr', 'YlGn', 'Reds', 'RdPu', 'Greens', 'YlGnBu', 'Purples', 'GnBu', 'Greys', 'YlOrRd', 'PuRd', 'Blues', 'PuBuGn', 'viridis', 'plasma', 'inferno', 'magma'), value='OrRd'), SelectionSlider(options=('OrRd', 'PuBu', 'BuPu', 'Oranges', 'BuGn', 'YlOrBr', 'YlGn', 'Reds', 'RdPu', 'Greens', 'YlGnBu', 'Purples', 'GnBu', 'Greys', 'YlOrRd', 'PuRd', 'Blues', 'PuBuGn', 'viridis', 'plasma', 'inferno', 'magma'), value='OrRd'))), Figure(axes=[Axis(label_location='end', num_ticks=10, scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale(), side='left'), ColorAxis(orientation='vertical', scale=ColorScale(scheme=u'OrRd'), side='right')], fig_margin={'top': 50, 'right': 100, 'left': 50, 'bottom': 70}, layout=Layout(min_width=u'125px'), marks=[Scatter(color=array([  176.4052346 ,   216.42095543,   314.29475384,   538.38407376,
         725.13987278,   627.41208479,   722.42092654,   707.28520571,
         696.96332053,   738.02317073,   752.42752784,   897.85487854,
         973.95865106,   986.12615271,  1030.51247598,  1063.87990872,
        1213.28781603,  1192.77198966,  1224.07875982,  1138.66918589,
         883.37020431,   948.73206385,  1035.17568374,   960.9591817 ,
        1187.9346441 ,  1042.49807664,  1047.07392837,  1028.35554336,
        1181.6334648 ,  1328.56934179,  1344.06408436,  1381.88033632,
        1293.10176156,  1095.02211473,  1060.2308998 ,  1075.86579671,
        1198.89486478,  1319.13284966,  1280.40016792,  1250.16989286,
        1145.31459636,  1003.31280264,   832.68578358,  1027.7633231 ,
         976.79810493,   932.99067476,   807.71113876,   885.46017434,
         724.07038959,   702.79636157,   613.24970545,   651.93995523,
         600.85944148,   482.79622306,   479.97800023,   522.81118728,
         529.46290952,   559.71009929,   496.27788993,   460.00377333,
         392.75772855,   356.8024124 ,   275.48778419,   102.85952396,
         120.60213818,    80.42404456,   -82.59579013,   -36.31756458,
        -127.04740102,  -121.85286144,   -48.94380522,   -36.04551415,
          77.89455431,   -45.58802773,    -5.35386361,   -73.8348727 ,
        -160.91458762,  -218.7995541 ,  -249.95480731,  -244.33827309,
        -360.85325717,  -270.77060847,  -224.2043645 ,  -377.82873313,
        -229.00351375,   -39.41459614,    78.46336097,    60.47087739,
         -46.60438476,    58.84078793,    18.52309324,   140.76760027,
         161.59509808,   259.25900173,   294.89564145,   365.55295827,
         366.60296034,   545.19000973,   557.881219  ,   598.08015534]), colors=['steelblue'], interactions={'hover': 'tooltip'}, scales={'y': LinearScale(), 'x': LinearScale(), 'color': ColorScale(scheme=u'OrRd')}, scales_metadata={'opacity': {'dimension': 'opacity'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'skew': {'dimension': 'skew'}, 'rotation': {'dimension': 'rotation'}, 'size': {'dimension': 'size'}}, stroke='black', tooltip_style={'opacity': 0.9}, x=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), y=array([  176.4052346 ,   216.42095543,   314.29475384,   538.38407376,
         725.13987278,   627.41208479,   722.42092654,   707.28520571,
         696.96332053,   738.02317073,   752.42752784,   897.85487854,
         973.95865106,   986.12615271,  1030.51247598,  1063.87990872,
        1213.28781603,  1192.77198966,  1224.07875982,  1138.66918589,
         883.37020431,   948.73206385,  1035.17568374,

## OrdinalColorScale

The `OrdinalColorScale` is a color scale for categorical data, i.e. data that does not have an intrinsic order.

The scale colors may be specified by the user, or chosen from a set scheme. As of now, the supported color schemes are the colorbrewer categorical schemes, listed here along with their maximum number of colors.

In [13]:
ord_schemes = {
    'Set2': 8,
    'Accent': 8,
    'Set1': 9,
    'Set3': 12,
    'Dark2': 8,
    'Paired': 12,
    'Pastel2': 8,
    'Pastel1': 9,
}

In [14]:
def partition(array, n_groups):
    n_elements = len(array)
    if n_groups > n_elements:
        return np.arange(n_elements)
    n_per_group = n_elements // n_groups + (n_elements % n_groups > 0)
    return np.tile(range(1, n_groups+1), n_per_group)[:n_elements]

In [15]:
# Define the control widgets
n_groups_slider = IntSlider(description='n colors', min=3)
scheme_slider = SelectionSlider(options=ord_schemes)
scheme_dd = Dropdown(description='Scheme',
                     options=ord_schemes)
link((scheme_dd, 'label'), (scheme_slider, 'label'))
    
def update_scheme(*args):
    col_sc.scheme = scheme_slider.label
    ax_c.label = scheme_slider.label
    n_groups_slider.max = scheme_slider.value

def update_categories(*args):
    groups = partition(names, n_groups_slider.value)
    market_map.color = groups
    market_map.groups = groups

n_groups_slider.observe(update_categories, 'value')
scheme_slider.observe(update_scheme, 'value')

# Define the bqplot marketmap
names = range(100)

col_sc = OrdinalColorScale()
ax_c = ColorAxis(scale=col_sc)

market_map = MarketMap(names=names,
                       display_text=['' for _ in names],
                       scales={'color': col_sc}, axes=[ax_c],
                       layout=Layout(min_width='800px', min_height='600px'))
update_scheme()
update_categories()

VBox([HBox([scheme_dd, scheme_slider, n_groups_slider]), market_map])

VBox(children=(HBox(children=(Dropdown(description=u'Scheme', options={'Pastel2': 8, 'Paired': 12, 'Pastel1': 9, 'Set1': 9, 'Set2': 8, 'Set3': 12, 'Dark2': 8, 'Accent': 8}, value=8), SelectionSlider(options={'Pastel2': 8, 'Paired': 12, 'Pastel1': 9, 'Set1': 9, 'Set2': 8, 'Set3': 12, 'Dark2': 8, 'Accent': 8}, value=8), IntSlider(value=3, description=u'n colors', max=8, min=3))), MarketMap(axes=[ColorAxis(label=u'Pastel2', scale=OrdinalColorScale(scheme=u'Pastel2'))], color=array([1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2,
       3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1,
       2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3,
       1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2,
       3, 1, 2, 3, 1, 2, 3, 1]), colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], display_text=array(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], 
      dtype='|S1'), groups=array([1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2,
       3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1,
       2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3,
       1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2,
       3, 1, 2, 3, 1, 2, 3, 1]), layout=Layout(min_height=u'600px', min_width=u'800px'), map_margin={'top': 50, 'right': 50, 'left': 50, 'bottom': 50}, names=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), scales={'color': OrdinalColorScale(scheme=u'Pastel2')})))